## Setup

In [1]:
import pandas as pd
import numpy as np
import re

## Load data

In [2]:
df = pd.read_csv('responses.csv')

df.head(1)

,Timestamp,Major(s),Minor(s) (N/A if none),Year,Gender,GPA,Are you Spanish/Hispanic/Latino?,What is your race?,Do you commute or live on campus?,Are you a transfer?,...,Sexuality,What type of clubs are you in?,What sports are you in? (N/A if you don't participate in sports),"Are you involved in Greek Life on campus (i.e., a member of a sorority or fraternity)?",Relationship Status (select the option most accurate to your status),Length of Your Longest Relationship (Has to be an number of months),Number of Romantic Involvements (N/A if not applicable),"If in a relationship, did you meet your current partner at UCSD?","If you are in a relationship, is it long distance? (N/A if not applicable)","How many times a week do you go out? (e.g., going to a party, going to the mall, hanging out with friends, etc.)"
0,11/4/2025 11:00:42,General Biology,NaN,Third-year,Female,3.69,No,White/Caucasian,On Campus,No,...,Heterosexual/Straight,"Professional (Consulting Club, Business Club, ...",Equestrian Team,No,Single,0,NaN,No,NaN,2


## Clean specific columns


In [3]:
def clean_gpa(gpa_value):
    if pd.isna(gpa_value) or gpa_value == 'N/A':
        return np.nan
    gpa_str = str(gpa_value).strip()
    match = re.search(r'([0-3]\.?\d*|4\.?0*)', gpa_str)
    if match:
        gpa_num = float(match.group(1))
        if 0 <= gpa_num <= 4.0:
            return round(gpa_num, 2)
    return np.nan

df['GPA'] = df['GPA'].apply(clean_gpa)

In [4]:
def clean_relationship_length(value):
    if pd.isna(value) or value == 'N/A':
        return np.nan
    value_str = str(value).strip().lower()
    value_str = re.sub(r'\s*months?\s*', '', value_str)
    match = re.search(r'(\d+)', value_str)
    if match:
        return int(match.group(1))
    return np.nan

rel_length_col = 'Length of Your Longest Relationship (Has to be an number of months)'
df['Relationship_Length_Months'] = df[rel_length_col].apply(clean_relationship_length)

In [5]:
def clean_times_per_week(value):
    if pd.isna(value) or value == 'N/A':
        return np.nan
    value_str = str(value).strip().lower()
    if 'once' in value_str or '1' in value_str[:2]:
        if 'twice' in value_str or '2' in value_str:
            return 1.5
        return 1
    elif 'twice' in value_str or value_str == '2':
        return 2
    elif 'everyday' in value_str or 'every day' in value_str:
        return 7
    elif 'daily' in value_str:
        return 7
    range_match = re.search(r'(\d+)\s*[-/]\s*(\d+)', value_str)
    if range_match:
        num1 = int(range_match.group(1))
        num2 = int(range_match.group(2))
        return (num1 + num2) / 2
    match = re.search(r'(\d+)', value_str)
    if match:
        num = int(match.group(1))
        return min(num, 7)
    return np.nan

times_col = 'How many times a week do you go out? (e.g., going to a party, going to the mall, hanging out with friends, etc.)'
df['Outing_Times_Per_Week'] = df[times_col].apply(clean_times_per_week)


In [6]:
def clean_romantic_involvements(value):
    if pd.isna(value) or value == 'N/A':
        return np.nan
    value_str = str(value).strip()
    match = re.search(r'(\d+)', value_str)
    if match:
        return int(match.group(1))
    return np.nan

romantic_col = 'Number of Romantic Involvements (N/A if not applicable)'
df['Romantic_Involvements'] = df[romantic_col].apply(clean_romantic_involvements)

## Standardize NaN values

In [7]:
df = df.replace(['N/A', 'n/a', 'N/a', 'None', 'none'], np.nan)
df = df.replace('', np.nan)

## Remove whitespace

In [8]:
text_columns = ['Major(s)', 'Minor(s) (N/A if none)', 'Year', 'Gender', 
                'Which college are you in?', 'What is your job status?', 
                'Sexuality', 'Relationship Status (select the option most accurate to your status)']

for col in text_columns:
    if col in df.columns:
        df[col] = df[col].astype(str).str.strip()
        df[col] = df[col].replace('nan', np.nan)

## Standardize answers

In [9]:
major_minor_mapping = {
    'business econ': 'Business Economics',
    'business economics': 'Business Economics',
    'math econ': 'Math and Economics',
    'math and economics': 'Math and Economics',
    'computer science': 'Computer Science',
    'data science': 'Data Science',
    'dsc': 'Data Science',
    'math cs': 'Math and Computer Science',
    'psychology': 'Psychology',
    'clinical psychology': 'Clinical Psychology',
    'biology': 'Biology',
    'general biology': 'General Biology',
    'cognitive science': 'Cognitive Science',
    'cogs': 'Cognitive Science',
    'cognitive science design': 'Cognitive Science Design',
    'cognitive science philosopy': 'Cognitive Science Philosophy',
    'business analytics': 'Business Analytics',
    'mathematics': 'Mathematics',
    'math': 'Mathematics',
    'neurobiology': 'Neurobiology',
    'marine science': 'Marine Science',
    'bioengineering bioinformatics': 'Bioengineering Bioinformatics',
    'molecular and cell biology': 'Molecular and Cell Biology',
    'theater': 'Theater',
    'geoscience': 'Geoscience',
    'literatures in english': 'Literatures in English',
    'finance': 'Finance',
    'marketing': 'Marketing',
    'astrophysics': 'Astrophysics',
    'bioethics': 'Bioethics',
    'environmental science': 'Environmental Science',
    'real estate development': 'Real Estate Development',
    'international studies': 'International Studies',
    'political science': 'Political Science'
}

def standardize_major_minor(value):
    if pd.isna(value) or str(value).lower() in ['n/a', 'none', 'nan']:
        return np.nan
    
    value_str = str(value).strip()
    value_lower = value_str.lower()
    
    value_clean = re.sub(r'\s*(b\.?s\.?|b\.?a\.?)\s*$', '', value_lower, flags=re.IGNORECASE)
    
    if ',' in value_str or '+' in value_str or ('-' in value_str and ' ' in value_str):
        parts = re.split(r'[,+]|(?<=\s)-(?=\s)', value_str)
        standardized_parts = []
        for part in parts:
            part = part.strip()
            if part:
                part_lower = part.lower()
                part_clean = re.sub(r'\s*(b\.?s\.?|b\.?a\.?)\s*$', '', part_lower, flags=re.IGNORECASE)
                standardized = major_minor_mapping.get(part_clean, part.title())
                standardized_parts.append(standardized)
        return ', '.join(standardized_parts)
    
    standardized = major_minor_mapping.get(value_clean, value_str.title())
    return standardized

df['Major(s)'] = df['Major(s)'].apply(standardize_major_minor)
df['Minor(s) (N/A if none)'] = df['Minor(s) (N/A if none)'].apply(standardize_major_minor)

## One-hot encoding


In [10]:
df['Length of Your Longest Relationship (Has to be an number of months)'] = df['Relationship_Length_Months']
df['How many times a week do you go out? (e.g., going to a party, going to the mall, hanging out with friends, etc.)'] = df['Outing_Times_Per_Week']
df['Number of Romantic Involvements (N/A if not applicable)'] = df['Romantic_Involvements']

df = df.drop(columns=['Relationship_Length_Months', 'Outing_Times_Per_Week', 'Romantic_Involvements']) 

In [11]:
clubs_col = 'What type of clubs are you in?'
club_types = ['Professional', 'Recreational', 'Athletic', 'Skill development', 'Cultural', 'Student council']

for club_type in club_types:
    df[f'Club_{club_type.replace(" ", "_")}'] = df[clubs_col].apply(
        lambda x: 1 if pd.notna(x) and club_type.lower() in str(x).lower() else 0
    )

df['Club_None'] = df[clubs_col].apply(
    lambda x: 1 if pd.isna(x) or str(x).lower() in ['none', 'n/a', 'nan'] else 0
)

In [12]:
sports_col = 'What sports are you in? (N/A if you don\'t participate in sports)'
df['Play_Sports'] = df[sports_col].apply(
    lambda x: 1 if pd.notna(x) and str(x).lower() not in ['n/a', 'none', 'nan'] else 0
)

In [13]:
binary_cols = {
    'Are you involved in Greek Life on campus (i.e., a member of a sorority or fraternity)?': 'Greek_Life',
    'Do you commute or live on campus?': 'Commute',
    'Are you a transfer?': 'Transfer',
    'Are you Spanish/Hispanic/Latino?': 'Hispanic_Latino',
    'If in a relationship, did you meet your current partner at UCSD?': 'Met_Partner_UCSD',
    'If you are in a relationship, is it long distance? (N/A if not applicable)': 'Long_Distance'
}

for col, prefix in binary_cols.items():
    if col in df.columns:
        if col == 'Do you commute or live on campus?':
            df[f'{prefix}'] = (df[col] == 'Commute').astype(int)
        else:
            df[f'{prefix}'] = (df[col] == 'Yes').astype(int)

In [14]:
categorical_cols = {
    'Relationship Status (select the option most accurate to your status)': 'Relationship_Status',
    'What is your job status?': 'Job_Status',
    'Sexuality': 'Sexuality',
    'Gender': 'Gender',
    'Year': 'Year',
    'Which college are you in?': 'College',
    'What is your race?': 'Race'
}

for col, prefix in categorical_cols.items():
    if col in df.columns:
        dummies = pd.get_dummies(df[col], prefix=prefix)
        df = pd.concat([df, dummies], axis=1)

## Drop original columns


In [15]:
cols_to_drop = [
    'Timestamp',
    'What type of clubs are you in?',
    'What sports are you in? (N/A if you don\'t participate in sports)',
    'Are you involved in Greek Life on campus (i.e., a member of a sorority or fraternity)?',
    'Do you commute or live on campus?',
    'Are you a transfer?',
    'Are you Spanish/Hispanic/Latino?',
    'If in a relationship, did you meet your current partner at UCSD?',
    'If you are in a relationship, is it long distance? (N/A if not applicable)',
    'Relationship Status (select the option most accurate to your status)',
    'What is your job status?',
    'Sexuality',
    'Gender',
    'Year',
    'Which college are you in?',
    'What is your race?'
]

df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

## Save cleaned data

In [16]:
df.to_csv('cleaned.csv', index=False)